# Web scraping do Insper

### Importações necessárias

In [ ]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import re

### Código para construir o csv com as matérias de determinado curso

In [ ]:
def procura_plano(curso):
    curso = curso.lower()
    materias = []

    for i in tqdm(range(51599, 37000, -1)):
        page = requests.get(f'https://cgi.insper.edu.br/DiarioDeClasse/PlanoDeAula/PlanoDeAulaPDF.aspx?t=Plano%20de%20Aula%20Atualizado&id={i},&d=false')
        soup = BeautifulSoup(page.content, "html.parser")
        elements = soup.find_all('div', 'textInline')
        for element in elements:
            texto = element.text.strip().lower()
            materia = elements[1].text.strip().lower()
            ano = elements[4].text.strip().split()[0]
            tipo = 'obrigatória'
            if re.search(r'_elet_', elements[2].text.strip().lower()):
                continue
            elif re.search(r'_opt_', elements[2].text.strip().lower()):
                continue
            
            if texto == curso and materia not in materias:
                materias.append(materia)
                with open('links.csv', 'a') as f:
                    f.write(f'{i},"{materia}","{curso}","{tipo}","https://cgi.insper.edu.br/DiarioDeClasse/PlanoDeAula/PlanoDeAulaPDF.aspx?t=Plano%20de%20Aula%20Atualizado&id={i},&d=false"\n')

In [ ]:
curso = input('Digite o nome do curso: ')
procura_plano(curso)

### O código abaixo serve para trabalhar com o csv construído

In [ ]:
planos = pd.read_csv('plano_de_aula.csv')

In [ ]:
materia = input('Qual matéria você quer pesquisar? ').lower()
plano_pesquisado = planos.loc[planos['Materia'] == materia, :]
plano_pesquisado.Link.values[0]